In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# 1. Предобработка

In [3]:
data = pd.read_csv("answer_data_result.csv", names=['question_num', 'link', 'num_answer', 'tags', 'is_question', 'answer_text', 'answer_author_name', 'answer_reputation', 'answer_likes'], header=None)
data

,question_num,link,num_answer,tags,is_question,answer_text,answer_author_name,answer_reputation,answer_likes
0,1,https://scifi.stackexchange.com/questions/6887...,0,"['dc', 'superman', 'character-identification',...",True,"In Man of Steel, when Zod attacks the Council,...",Kreann,8407,13
1,1,https://scifi.stackexchange.com/questions/6887...,1,"['dc', 'superman', 'character-identification',...",False,This is Nam-Ek\n\nNam-Ek was born on the dista...,EricSSH,1383,13
2,1,https://scifi.stackexchange.com/questions/6887...,2,"['dc', 'superman', 'character-identification',...",False,"This could also be Tor-An, a large Kryptonian ...",Omegacron,633000,4
3,1,https://scifi.stackexchange.com/questions/6887...,3,"['dc', 'superman', 'character-identification',...",False,That was Sciencist NON....the mentor /ex-frien...,CHRISTOPHER A. DAVIS DAVIS,1,-1
4,2,https://scifi.stackexchange.com/questions/1614...,0,['mummy-2017'],True,"In The Mummy 2017, the title character has sym...",CodeMed,108000,12
...,...,...,...,...,...,...,...,...,...
62999,17998,https://scifi.stackexchange.com/questions/1412...,6,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",False,Want to improve this answer? Add details and i...,Tim,6237,1
63000,17999,https://scifi.stackexchange.com/questions/1342...,0,"['harry-potter', 'voldemort']",True,Is there verification either in canon or from ...,Slytherincess,165000,5
63001,17999,https://scifi.stackexchange.com/questions/1342...,1,"['harry-potter', 'voldemort']",False,Christiana: How did voldemort get his wand bac...,DVK-on-Ahch-To,343000,2
63002,18000,https://scifi.stackexchange.com/questions/1703...,0,"['star-trek', 'star-trek-discovery']",True,In the premiere episode of Star Trek: Discover...,Politank-Z,141000,11


In [4]:
data['num_answer'].value_counts()

num_answer
0     17993
1     17745
2     10768
3      6265
4      3724
5      2259
6      1431
7       920
8       607
9       395
10      273
11      190
12      132
13       97
14       66
15       46
16       32
17       22
18       19
19        6
20        3
21        2
22        2
23        2
24        2
25        1
26        1
27        1
Name: count, dtype: int64

In [3]:
data_head = data.head(12)
data_head

,question_num,link,num_answer,tags,is_question,answer_text,answer_author_name,answer_reputation,answer_likes
0,1,https://scifi.stackexchange.com/questions/6887...,0,"['dc', 'superman', 'character-identification',...",True,"In Man of Steel, when Zod attacks the Council,...",Kreann,8407,13
1,1,https://scifi.stackexchange.com/questions/6887...,1,"['dc', 'superman', 'character-identification',...",False,This is Nam-Ek\n\nNam-Ek was born on the dista...,EricSSH,1383,13
2,1,https://scifi.stackexchange.com/questions/6887...,2,"['dc', 'superman', 'character-identification',...",False,"This could also be Tor-An, a large Kryptonian ...",Omegacron,633000,4
3,1,https://scifi.stackexchange.com/questions/6887...,3,"['dc', 'superman', 'character-identification',...",False,That was Sciencist NON....the mentor /ex-frien...,CHRISTOPHER A. DAVIS DAVIS,1,-1
4,2,https://scifi.stackexchange.com/questions/1614...,0,['mummy-2017'],True,"In The Mummy 2017, the title character has sym...",CodeMed,108000,12
5,2,https://scifi.stackexchange.com/questions/1614...,1,['mummy-2017'],False,I asked my good friend Elizabeth (Lizzie) Yian...,Valorum,695000,21
6,2,https://scifi.stackexchange.com/questions/1614...,2,['mummy-2017'],False,The scriptures on Princess Ahmanet's face (Sof...,Kurumi83,31,3
7,2,https://scifi.stackexchange.com/questions/1614...,3,['mummy-2017'],False,"It reminds me rather on the ""Berber Alphabet"" ...",Bruder Lustig,111,-2
8,3,https://scifi.stackexchange.com/questions/2790...,0,"['story-identification', 'short-stories', 'ali...",True,I've been trying to find this story for years....,Ralph Johnson Jr.,163,16
9,3,https://scifi.stackexchange.com/questions/2790...,1,"['story-identification', 'short-stories', 'ali...",False,"""The Pillows"", a short story by Margaret St. C...",user14111,164000,24


## 1.0 Не запускать

In [ ]:
A = data[data['num_answer'] == 5]
A

,question_num,link,num_answer,tags,is_question,answer_text,answer_author_name,answer_reputation,answer_likes
15,4,https://scifi.stackexchange.com/questions/1211...,5,"['star-wars', 'clones']",False,I do not know why the First Order and/or the N...,Nigel,1,-1
26,6,https://scifi.stackexchange.com/questions/1403...,5,['star-trek'],False,Marla McGivers identifies Khan Singh as being ...,Valorum,695000,29
54,15,https://scifi.stackexchange.com/questions/5204...,5,"['star-trek', 'star-trek-tos']",False,The warp core is in The Original Series Enterp...,answerer,21,2
97,28,https://scifi.stackexchange.com/questions/1146...,5,"['star-wars', 'darth-vader', 'return-of-the-je...",False,The Emperor also could have sensed Luke's grea...,EvilSnack,3742,1
120,35,https://scifi.stackexchange.com/questions/2758...,5,"['dc', 'nolan-batman-trilogy', 'the-dark-knigh...",False,"Looking back now at the trilogy, there is an o...",Clint,19,1
...,...,...,...,...,...,...,...,...,...
20810,5948,https://scifi.stackexchange.com/questions/6923...,5,"['hard-sci-fi', 'gravity']",False,"Not as much gravity as a neutron star, but Iai...",Robert,2007,1
20858,5961,https://scifi.stackexchange.com/questions/9339...,5,"['star-trek', 'star-trek-tng', 'behind-the-sce...",False,"I'm not sure about official casting reasons, b...",DVK-on-Ahch-To,343000,3
20904,5975,https://scifi.stackexchange.com/questions/1470...,5,"['star-trek', 'star-trek-tng', 'star-trek-ds9'...",False,The whole point of synthahol is that you can c...,Chris Johns,3198,3
20919,5980,https://scifi.stackexchange.com/questions/8564...,5,"['star-trek', 'star-trek-enterprise']",False,I always just assumed that the warp field did ...,Gerry,1,-3


In [ ]:
import random
idxs = list(A['question_num'])
idxs = random.sample(idxs, 200)
len(idxs)

200

In [ ]:
A0 = data[data['num_answer'] == 0]
A0 = A0.loc[A0['question_num'].isin(idxs)]
A0['answer_text']

115      In The Dark Knight Rises it is made clear that...
313      So a bunch of kids all of a sudden "go missing...
391      "If you're not with me then you're my enemy"\n...
433      As we all know that Voldemort had one of his H...
509      It's because he did not die in the alternate t...
                               ...                        
20744    During the battle of Scarif when Jyn is trying...
20805    I've just started reading Foundation and Gaal ...
20853    I always find it a little jarring that Picard ...
20914    When the enterprise (ENT) is moving at warp we...
20973    Worf was rescued from the wreckage of the Khit...
Name: answer_text, Length: 200, dtype: object

In [ ]:
A1 = data[data['num_answer'] == 1]
A1 = A1.loc[A1['question_num'].isin(idxs)]
A1['answer_text']

116      The film ["Batman Begins" ] takes place in 200...
314      He didn't leave school to go to Hogwarts. He'd...
392      Given that one of the main points of the "Rule...
434      For a start, Harry was protected from Voldemor...
510      In S4E2, Walter mentioned that Olivia was kidn...
                               ...                        
20745    From the novelization, these are the Imperial ...
20806    Probably Hexanerax 2 in Childhood's End by Art...
20854    Bob Justman, a member of the Star Trek: The Ne...
20915    Main Canon\nThere are several instances within...
20974    I suggest that Worf doesn't act very Klingon a...
Name: answer_text, Length: 200, dtype: object

In [ ]:
A2 = data[data['num_answer'] == 2]
A2 = A2.loc[A2['question_num'].isin(idxs)]

A3 = data[data['num_answer'] == 3]
A3 = A3.loc[A3['question_num'].isin(idxs)]

A4 = data[data['num_answer'] == 4]
A4 = A4.loc[A4['question_num'].isin(idxs)]

A5 = data[data['num_answer'] == 5]
A5 = A5.loc[A5['question_num'].isin(idxs)]

In [ ]:
A2['answer_text']

117      I'm broadly in agreement with the two existing...
315      You write, "I assume that Hogwarts isn't an of...
393      The Rule of Two\n\nThe Sith "Rule of Two" was ...
435      The Dark Lord knew it was too well protected.\...
511      Maybe, just maybe, Walternate replaced Broyles...
                               ...                        
20746    Could project black-saber be referring to the ...
20807    I'll venture that the winner is none other tha...
20855    The reason there was no public outcry in Franc...
20916    Richard's answer is a good one.\nBut I want to...
20975    Five is a considerable number of years to be i...
Name: answer_text, Length: 200, dtype: object

In [ ]:
df = pd.DataFrame()
df['question'] = A0['answer_text']
df['answer_1'] = list(A1['answer_text'])
df['answer_2'] = list(A2['answer_text'])
df['answer_3'] = list(A3['answer_text'])
df['answer_4'] = list(A4['answer_text'])
df['answer_5'] = list(A5['answer_text'])
df = df.reset_index(drop=True)
df

,question,answer_1,answer_2,answer_3,answer_4,answer_5
0,In The Dark Knight Rises it is made clear that...,"The film [""Batman Begins"" ] takes place in 200...",I'm broadly in agreement with the two existing...,Batman Begins: 2007\nThe Dark Knight: 2008\nTh...,When Bruce is looking at Selina Kyle's file in...,"Looking back now at the trilogy, there is an o..."
1,"So a bunch of kids all of a sudden ""go missing...",He didn't leave school to go to Hogwarts. He'd...,"You write, ""I assume that Hogwarts isn't an of...","10,000 children are 'missing' from the English...","As far as the government is concerned, Harry a...","In one of the books, though I can't remember w..."
2,"""If you're not with me then you're my enemy""\n...","Given that one of the main points of the ""Rule...","The Rule of Two\n\nThe Sith ""Rule of Two"" was ...",Anakin didn't want to kill Obi-Wan. He fully e...,"First, there's no reason to assume Anakin even...","If so, Anakin and Obi Wan would kill Palpatine..."
3,As we all know that Voldemort had one of his H...,"For a start, Harry was protected from Voldemor...",The Dark Lord knew it was too well protected.\...,The most powerful spell in this world of wizar...,Because his mother died to protect him and 'wh...,If Voldemort could easily look into the future...
4,It's because he did not die in the alternate t...,"In S4E2, Walter mentioned that Olivia was kidn...","Maybe, just maybe, Walternate replaced Broyles...",Go to http://tvovermind.zap2it.com/fox/fringe/...,"This really bothers me because either way, Fau...",How about this: what if Fauxlivia stayed Over ...
...,...,...,...,...,...,...
195,During the battle of Scarif when Jyn is trying...,"From the novelization, these are the Imperial ...",Could project black-saber be referring to the ...,"For full explanations, check out Star Wars Exp...",A couple seem to refer to non canonical elemen...,"Well actually, there's a good chance that proj..."
196,I've just started reading Foundation and Gaal ...,Probably Hexanerax 2 in Childhood's End by Art...,I'll venture that the winner is none other tha...,I don't remember the name of the story; but yo...,By far not the one with the highest density (l...,"Not as much gravity as a neutron star, but Iai..."
197,I always find it a little jarring that Picard ...,"Bob Justman, a member of the Star Trek: The Ne...",The reason there was no public outcry in Franc...,"So I am Francophone, and this is how I see it....",I don't think the Universal Translator is nece...,"I'm not sure about official casting reasons, b..."
198,When the enterprise (ENT) is moving at warp we...,Main Canon\nThere are several instances within...,Richard's answer is a good one.\nBut I want to...,The only reason for stars to streak past at th...,TV audiences need some visual indication that ...,I always just assumed that the warp field did ...


In [ ]:
df.to_json("200_questions.json")

In [ ]:
pd.read_json('filename.json')

,question,answer_1,answer_2,answer_3,answer_4,answer_5
0,In The Dark Knight Rises it is made clear that...,"The film [""Batman Begins"" ] takes place in 200...",I'm broadly in agreement with the two existing...,Batman Begins: 2007\nThe Dark Knight: 2008\nTh...,When Bruce is looking at Selina Kyle's file in...,"Looking back now at the trilogy, there is an o..."
1,"So a bunch of kids all of a sudden ""go missing...",He didn't leave school to go to Hogwarts. He'd...,"You write, ""I assume that Hogwarts isn't an of...","10,000 children are 'missing' from the English...","As far as the government is concerned, Harry a...","In one of the books, though I can't remember w..."
2,"""If you're not with me then you're my enemy""\n...","Given that one of the main points of the ""Rule...","The Rule of Two\n\nThe Sith ""Rule of Two"" was ...",Anakin didn't want to kill Obi-Wan. He fully e...,"First, there's no reason to assume Anakin even...","If so, Anakin and Obi Wan would kill Palpatine..."
3,As we all know that Voldemort had one of his H...,"For a start, Harry was protected from Voldemor...",The Dark Lord knew it was too well protected.\...,The most powerful spell in this world of wizar...,Because his mother died to protect him and 'wh...,If Voldemort could easily look into the future...
4,It's because he did not die in the alternate t...,"In S4E2, Walter mentioned that Olivia was kidn...","Maybe, just maybe, Walternate replaced Broyles...",Go to http://tvovermind.zap2it.com/fox/fringe/...,"This really bothers me because either way, Fau...",How about this: what if Fauxlivia stayed Over ...
...,...,...,...,...,...,...
195,During the battle of Scarif when Jyn is trying...,"From the novelization, these are the Imperial ...",Could project black-saber be referring to the ...,"For full explanations, check out Star Wars Exp...",A couple seem to refer to non canonical elemen...,"Well actually, there's a good chance that proj..."
196,I've just started reading Foundation and Gaal ...,Probably Hexanerax 2 in Childhood's End by Art...,I'll venture that the winner is none other tha...,I don't remember the name of the story; but yo...,By far not the one with the highest density (l...,"Not as much gravity as a neutron star, but Iai..."
197,I always find it a little jarring that Picard ...,"Bob Justman, a member of the Star Trek: The Ne...",The reason there was no public outcry in Franc...,"So I am Francophone, and this is how I see it....",I don't think the Universal Translator is nece...,"I'm not sure about official casting reasons, b..."
198,When the enterprise (ENT) is moving at warp we...,Main Canon\nThere are several instances within...,Richard's answer is a good one.\nBut I want to...,The only reason for stars to streak past at th...,TV audiences need some visual indication that ...,I always just assumed that the warp field did ...


## 1.1 Предобработка

In [4]:
data = pd.read_csv("answer_data_result.csv", names=['question_num', 'link', 'num_answer', 'tags', 'is_question', 'answer_text', 'answer_author_name', 'answer_reputation', 'answer_likes'], header=None)
data

,question_num,link,num_answer,tags,is_question,answer_text,answer_author_name,answer_reputation,answer_likes
0,1,https://scifi.stackexchange.com/questions/6887...,0,"['dc', 'superman', 'character-identification',...",True,"In Man of Steel, when Zod attacks the Council,...",Kreann,8407,13
1,1,https://scifi.stackexchange.com/questions/6887...,1,"['dc', 'superman', 'character-identification',...",False,This is Nam-Ek\n\nNam-Ek was born on the dista...,EricSSH,1383,13
2,1,https://scifi.stackexchange.com/questions/6887...,2,"['dc', 'superman', 'character-identification',...",False,"This could also be Tor-An, a large Kryptonian ...",Omegacron,633000,4
3,1,https://scifi.stackexchange.com/questions/6887...,3,"['dc', 'superman', 'character-identification',...",False,That was Sciencist NON....the mentor /ex-frien...,CHRISTOPHER A. DAVIS DAVIS,1,-1
4,2,https://scifi.stackexchange.com/questions/1614...,0,['mummy-2017'],True,"In The Mummy 2017, the title character has sym...",CodeMed,108000,12
...,...,...,...,...,...,...,...,...,...
20983,5999,https://scifi.stackexchange.com/questions/8642...,1,"['story-identification', 'episode-identificati...",False,Thanks to a comment by @Hypnosifl I have disco...,Frank Zafka,245,4
20984,5999,https://scifi.stackexchange.com/questions/8642...,2,"['story-identification', 'episode-identificati...",False,"Assuming that the timeframe is correct, and th...",Omegacron,633000,2
20985,5999,https://scifi.stackexchange.com/questions/8642...,3,"['story-identification', 'episode-identificati...",False,It was possibly don’t be afraid of the dark th...,Dylan,1,-1
20986,6000,https://scifi.stackexchange.com/questions/1983...,0,['story-identification'],True,"I read a story a while ago about humans, as a ...",Story searcher 38373635343,221,22


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63004 entries, 0 to 63003
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   question_num        63004 non-null  int64 
 1   link                63004 non-null  object
 2   num_answer          63004 non-null  int64 
 3   tags                63004 non-null  object
 4   is_question         63004 non-null  bool  
 5   answer_text         63004 non-null  object
 6   answer_author_name  63003 non-null  object
 7   answer_reputation   63004 non-null  int64 
 8   answer_likes        63004 non-null  int64 
dtypes: bool(1), int64(4), object(4)
memory usage: 3.9+ MB


In [ ]:
df = data.copy()
df

,question_num,link,num_answer,tags,is_question,answer_text,answer_author_name,answer_reputation,answer_likes
0,1,https://scifi.stackexchange.com/questions/6887...,0,"['dc', 'superman', 'character-identification',...",True,"In Man of Steel, when Zod attacks the Council,...",Kreann,8407,13
1,1,https://scifi.stackexchange.com/questions/6887...,1,"['dc', 'superman', 'character-identification',...",False,This is Nam-Ek\n\nNam-Ek was born on the dista...,EricSSH,1383,13
2,1,https://scifi.stackexchange.com/questions/6887...,2,"['dc', 'superman', 'character-identification',...",False,"This could also be Tor-An, a large Kryptonian ...",Omegacron,633000,4
3,1,https://scifi.stackexchange.com/questions/6887...,3,"['dc', 'superman', 'character-identification',...",False,That was Sciencist NON....the mentor /ex-frien...,CHRISTOPHER A. DAVIS DAVIS,1,-1
4,2,https://scifi.stackexchange.com/questions/1614...,0,['mummy-2017'],True,"In The Mummy 2017, the title character has sym...",CodeMed,108000,12
...,...,...,...,...,...,...,...,...,...
62999,17998,https://scifi.stackexchange.com/questions/1412...,6,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",False,Want to improve this answer? Add details and i...,Tim,6237,1
63000,17999,https://scifi.stackexchange.com/questions/1342...,0,"['harry-potter', 'voldemort']",True,Is there verification either in canon or from ...,Slytherincess,165000,5
63001,17999,https://scifi.stackexchange.com/questions/1342...,1,"['harry-potter', 'voldemort']",False,Christiana: How did voldemort get his wand bac...,DVK-on-Ahch-To,343000,2
63002,18000,https://scifi.stackexchange.com/questions/1703...,0,"['star-trek', 'star-trek-discovery']",True,In the premiere episode of Star Trek: Discover...,Politank-Z,141000,11


In [ ]:
df['question_author'] = ''
df['question_text'] = ''
df['question_author_reputation'] = 0
df['question_author_likes'] = 0

In [ ]:
q = data[data['is_question'] == True]
#q = q.reset_index()
q.index = list(q.question_num)
#q = q.drop(columns=['index'])
q

,question_num,link,num_answer,tags,is_question,answer_text,answer_author_name,answer_reputation,answer_likes
1,1,https://scifi.stackexchange.com/questions/6887...,0,"['dc', 'superman', 'character-identification',...",True,"In Man of Steel, when Zod attacks the Council,...",Kreann,8407,13
2,2,https://scifi.stackexchange.com/questions/1614...,0,['mummy-2017'],True,"In The Mummy 2017, the title character has sym...",CodeMed,108000,12
3,3,https://scifi.stackexchange.com/questions/2790...,0,"['story-identification', 'short-stories', 'ali...",True,I've been trying to find this story for years....,Ralph Johnson Jr.,163,16
4,4,https://scifi.stackexchange.com/questions/1211...,0,"['star-wars', 'clones']",True,With reference to Why did the Empire discontin...,thegreatjedi,342000,6
5,5,https://scifi.stackexchange.com/questions/1204...,0,"['aliens', 'history-of', 'terminology']",True,"It is fairly common nowadays, for example in S...",ThePopMachine,597000,2
...,...,...,...,...,...,...,...,...,...
17996,17996,https://scifi.stackexchange.com/questions/1098...,0,"['star-trek', 'star-trek-tng']",True,"In S3E09 of ST:TNG The Vengeance Factor, Riker...",Pᴀᴜʟsᴛᴇʀ2,1609,1
17997,17997,https://scifi.stackexchange.com/questions/1117...,0,[],True,Title says it all: With regards to the the on-...,Giacomo1968,171000,14
17998,17998,https://scifi.stackexchange.com/questions/1412...,0,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",True,"These aren't the droids you're looking for, yo...",King of NES,5466,48
17999,17999,https://scifi.stackexchange.com/questions/1342...,0,"['harry-potter', 'voldemort']",True,Is there verification either in canon or from ...,Slytherincess,165000,5


In [ ]:
for i in range (63004):
    n = df['question_num'][i]
    df['question_author'][i] = q['answer_author_name'][n]
    df['question_text'][i] = q['answer_text'][n]
    df['question_author_reputation'][i] = q['answer_reputation'][n]
    df['question_author_likes'][i] = q['answer_likes'][n]

In [ ]:
df.columns

Index(['question_num', 'link', 'num_answer', 'tags', 'is_question',
       'answer_text', 'answer_author_name', 'answer_reputation',
       'answer_likes', 'question_author', 'question_text',
       'question_author_reputation', 'question_author_likes'],
      dtype='object')

In [ ]:
data_final = df[['question_num', 'is_question', 'link', 'tags',
                 'question_author', 'question_text', 'question_author_reputation', 'question_author_likes',
                 'answer_author_name', 'answer_text', 'answer_reputation', 'answer_likes', 'num_answer']]
data_final = data_final[data_final['is_question'] == False]
data_final = data_final.reset_index()
data_final = data_final.drop(columns=['is_question', 'index'])
data_final

,question_num,link,tags,question_author,question_text,question_author_reputation,question_author_likes,answer_author_name,answer_text,answer_reputation,answer_likes,num_answer
0,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,EricSSH,This is Nam-Ek\n\nNam-Ek was born on the dista...,1383,13,1
1,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,Omegacron,"This could also be Tor-An, a large Kryptonian ...",633000,4,2
2,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,CHRISTOPHER A. DAVIS DAVIS,That was Sciencist NON....the mentor /ex-frien...,1,-1,3
3,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Valorum,I asked my good friend Elizabeth (Lizzie) Yian...,695000,21,1
4,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Kurumi83,The scriptures on Princess Ahmanet's face (Sof...,31,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,The Spooniest,"At least as of the first movie, Obi-Wan can't ...",3166,8,4
45007,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Peter Cordes,"Obi-Wan has the power to persuade minds, so wh...",525,3,5
45008,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Tim,Want to improve this answer? Add details and i...,6237,1,6
45009,17999,https://scifi.stackexchange.com/questions/1342...,"['harry-potter', 'voldemort']",Slytherincess,Is there verification either in canon or from ...,165000,5,DVK-on-Ahch-To,Christiana: How did voldemort get his wand bac...,343000,2,1


In [ ]:
sci_fi_data_final = data_final.copy()

In [ ]:
pd.DataFrame(sci_fi_data_final).to_csv("sci_fi_data_final.csv", index=False, header=False)

# 2. Text2Vec

In [5]:
import string
from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [6]:
data = pd.read_csv('sci_fi_data_final.csv', names=['question_num', 'link', 'tags',
                                                   'question_author', 'question_text', 'question_author_reputation', 'question_author_likes',
                                                   'answer_author_name', 'answer_text', 'answer_reputation', 'answer_likes', 'num_answer'], header=None)
data

,question_num,link,tags,question_author,question_text,question_author_reputation,question_author_likes,answer_author_name,answer_text,answer_reputation,answer_likes,num_answer
0,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,EricSSH,This is Nam-Ek\n\nNam-Ek was born on the dista...,1383,13,1
1,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,Omegacron,"This could also be Tor-An, a large Kryptonian ...",633000,4,2
2,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,CHRISTOPHER A. DAVIS DAVIS,That was Sciencist NON....the mentor /ex-frien...,1,-1,3
3,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Valorum,I asked my good friend Elizabeth (Lizzie) Yian...,695000,21,1
4,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Kurumi83,The scriptures on Princess Ahmanet's face (Sof...,31,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,The Spooniest,"At least as of the first movie, Obi-Wan can't ...",3166,8,4
45007,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Peter Cordes,"Obi-Wan has the power to persuade minds, so wh...",525,3,5
45008,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Tim,Want to improve this answer? Add details and i...,6237,1,6
45009,17999,https://scifi.stackexchange.com/questions/1342...,"['harry-potter', 'voldemort']",Slytherincess,Is there verification either in canon or from ...,165000,5,DVK-on-Ahch-To,Christiana: How did voldemort get his wand bac...,343000,2,1


In [7]:
data_head = data.head(12)
data_head

,question_num,link,tags,question_author,question_text,question_author_reputation,question_author_likes,answer_author_name,answer_text,answer_reputation,answer_likes,num_answer
0,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,EricSSH,This is Nam-Ek\n\nNam-Ek was born on the dista...,1383,13,1
1,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,Omegacron,"This could also be Tor-An, a large Kryptonian ...",633000,4,2
2,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,CHRISTOPHER A. DAVIS DAVIS,That was Sciencist NON....the mentor /ex-frien...,1,-1,3
3,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Valorum,I asked my good friend Elizabeth (Lizzie) Yian...,695000,21,1
4,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Kurumi83,The scriptures on Princess Ahmanet's face (Sof...,31,3,2
5,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Bruder Lustig,"It reminds me rather on the ""Berber Alphabet"" ...",111,-2,3
6,3,https://scifi.stackexchange.com/questions/2790...,"['story-identification', 'short-stories', 'ali...",Ralph Johnson Jr.,I've been trying to find this story for years....,163,16,user14111,"""The Pillows"", a short story by Margaret St. C...",164000,24,1
7,4,https://scifi.stackexchange.com/questions/1211...,"['star-wars', 'clones']",thegreatjedi,With reference to Why did the Empire discontin...,342000,6,Null,"Out-of-universe, Pablo Hidalgo explained that...",699000,5,1
8,4,https://scifi.stackexchange.com/questions/1211...,"['star-wars', 'clones']",thegreatjedi,With reference to Why did the Empire discontin...,342000,6,TheIronCheek,Wookieepedia says this (emphasis mine):\n\nAs ...,134000,3,2
9,4,https://scifi.stackexchange.com/questions/1211...,"['star-wars', 'clones']",thegreatjedi,With reference to Why did the Empire discontin...,342000,6,LoverofPhysics,"If we are going with Legends, they were ended ...",350,0,3


In [9]:
data_head = data_head[data_head['question_num'] == 4]
data_head['question_text'][7]

"With reference to Why did the Empire discontinue the clone trooper program?\nSame question but strictly no Legends. I want to know the reason within the Disney canon's context, as a lot of things with respect to the clones are different.\nEdit: I guess I should have made it clear. I know perfectly well existing clones age quickly. But why did they replace clones with birth born instead of new clones? Why is the clone trooper program discontinued?"

In [11]:
data_head['answer_text'][7]

'Out-of-universe, Pablo Hidalgo  explained that the production of clones was phased out at the end of the Clone Wars, and since the clones age at twice the rate of normal humans they are too old to serve as stormtroopers. They have been replaced with non-clone volunteers who are "fervently patriotic and loyal to the Empire". Video proof can be found on Youtube (starting at about 2:56 into the video).\nIn-universe, the canon novel Tarkin supports Hidalgo\'s claim that the clones are too old in this incident in which stormtroopers are loading Darth Vader\'s meditation chamber onto Tarkin\'s ship:\n\nWhen the stormtrooper operating the equipment accidentally allowed the\n  flattened sphere to bang against the edge of the cargo hold’s\n  retracted hatch, Vader stamped forward with his gloved hands clenched.\n“I warned you to be careful!” he shouted up at the trooper.\n“My apologies, Lord Vader. Wind shear from—”\n“Excuses won’t suffice, Sergeant Crest,” Vader cut him off. “Perhaps you are 

In [14]:
data_head['answer_text'][8]

'Wookieepedia says this (emphasis mine):\n\nAs stormtroopers, clones obeyed the new Emperor\'s will, enforced his laws, and destroyed the enemies of the Empire during the early years of the dark times. In time, however, the clones\' fighting skills deteriorated as a result of their accelerated aging process. Ultimately, clones were gradually replaced by birth-born Human recruits who, like their predecessors, were trained for absolute loyalty and obedience. Four years before the Battle of Yavin, all clones had been decommissioned from the Imperial Army. Nonetheless, a few clones remained in Imperial service acting as training instructors for the next generation of stormtroopers.\n\nThe Wookieepedia article sites these Disney canon sources:\n\nStar Wars Databank - Stormtroopers\nA New Dawn\nSDCC 2014: Star Wars: A New Dawn Panel - Liveblog on StarWars.com \nRebels Recon: Inside "The Lost Commanders" on StarWars.com\n\nEdit:\nTo answer your question about why clone production ceased, no D

In [13]:
data_head['answer_text'][9]

'If we are going with Legends, they were ended because of the Battle of Kamino, where clones attempted an uprising on the planet, but were quickly finished by Darth Vader, and his men. \nIn terms of Disney canon, due to the costs of continuing maintaining search an army, they were also fazed out in favor of humans; additionally, clones did age quickly, something that is exemplified by the *Star Wars Rebels * when we see Captain Rex, and his two Clone buddies. \nTo put it simply, clones were just a short term solution, to a short term problem. They were intended, per Star Wars The Clones Wars, among other sources to be able to be the most powerful army due to them inheriting the skills of Jango Fett, the best bounty hunter in the galaxy; nevertheless, as time went on, they evidently began to be ineffective, and human recruits were added in.'

In [4]:
Qst = data['question_text']
Qst = list(Qst)
len(Qst)

45011

In [5]:
Ans = data['answer_text']
Ans = list(Ans)
len(Ans)

45011

In [6]:
pip install -U text2vec

In [8]:
from text2vec import SentenceModel

model = SentenceModel('shibing624/text2vec-base-multilingual')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/496 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

2024-06-19 10:38:39.078 | DEBUG    | text2vec.sentence_model:__init__:80 - Use device: cpu


вопросы

In [9]:
sentences = Qst
embeddings = model.encode(sentences)
len(embeddings)

45011

In [10]:
q_cols = []
for i in range (384):
    c = 'Q' + str(i)
    q_cols.append(c)

In [11]:
Q = pd.DataFrame(embeddings)
Q.columns = q_cols
Q

,Q0,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q374,Q375,Q376,Q377,Q378,Q379,Q380,Q381,Q382,Q383
0,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,-0.041208,0.143133,...,0.238037,-0.010470,0.072340,-0.078872,-0.186629,-0.005452,-0.073400,0.008695,-0.076050,0.240609
1,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,-0.041208,0.143133,...,0.238037,-0.010470,0.072340,-0.078872,-0.186629,-0.005452,-0.073400,0.008695,-0.076050,0.240609
2,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,-0.041208,0.143133,...,0.238037,-0.010470,0.072340,-0.078872,-0.186629,-0.005452,-0.073400,0.008695,-0.076050,0.240609
3,0.003884,0.167487,-0.091520,0.130714,-0.096572,-0.141100,0.386765,-0.116209,0.060934,0.096892,...,-0.113742,-0.015066,0.101503,0.160942,0.227277,0.256956,0.230749,0.054927,0.010504,-0.065557
4,0.003884,0.167487,-0.091520,0.130714,-0.096572,-0.141100,0.386765,-0.116209,0.060934,0.096892,...,-0.113742,-0.015066,0.101503,0.160942,0.227277,0.256956,0.230749,0.054927,0.010504,-0.065557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,-0.073689,0.370423,-0.106119,-0.007474,-0.050272,-0.202091,0.404839,0.090825,0.175747,0.166854,...,0.282179,-0.152177,0.219008,-0.175435,-0.491605,0.331040,0.248201,-0.064353,0.101061,0.182705
45007,-0.073689,0.370423,-0.106119,-0.007474,-0.050272,-0.202091,0.404839,0.090825,0.175747,0.166854,...,0.282179,-0.152177,0.219008,-0.175435,-0.491605,0.331040,0.248201,-0.064353,0.101061,0.182705
45008,-0.073689,0.370423,-0.106119,-0.007474,-0.050272,-0.202091,0.404839,0.090825,0.175747,0.166854,...,0.282179,-0.152177,0.219008,-0.175435,-0.491605,0.331040,0.248201,-0.064353,0.101061,0.182705
45009,-0.148996,0.454998,-0.168628,0.024398,-0.065070,-0.111850,0.150651,-0.066592,0.160821,-0.228624,...,0.111209,0.093540,0.125449,-0.237482,-0.396527,-0.079588,0.019276,-0.100971,0.022527,-0.063107


In [12]:
Q_copy = Q.copy()
Q_copy

,Q0,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q374,Q375,Q376,Q377,Q378,Q379,Q380,Q381,Q382,Q383
0,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,-0.041208,0.143133,...,0.238037,-0.010470,0.072340,-0.078872,-0.186629,-0.005452,-0.073400,0.008695,-0.076050,0.240609
1,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,-0.041208,0.143133,...,0.238037,-0.010470,0.072340,-0.078872,-0.186629,-0.005452,-0.073400,0.008695,-0.076050,0.240609
2,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,-0.041208,0.143133,...,0.238037,-0.010470,0.072340,-0.078872,-0.186629,-0.005452,-0.073400,0.008695,-0.076050,0.240609
3,0.003884,0.167487,-0.091520,0.130714,-0.096572,-0.141100,0.386765,-0.116209,0.060934,0.096892,...,-0.113742,-0.015066,0.101503,0.160942,0.227277,0.256956,0.230749,0.054927,0.010504,-0.065557
4,0.003884,0.167487,-0.091520,0.130714,-0.096572,-0.141100,0.386765,-0.116209,0.060934,0.096892,...,-0.113742,-0.015066,0.101503,0.160942,0.227277,0.256956,0.230749,0.054927,0.010504,-0.065557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,-0.073689,0.370423,-0.106119,-0.007474,-0.050272,-0.202091,0.404839,0.090825,0.175747,0.166854,...,0.282179,-0.152177,0.219008,-0.175435,-0.491605,0.331040,0.248201,-0.064353,0.101061,0.182705
45007,-0.073689,0.370423,-0.106119,-0.007474,-0.050272,-0.202091,0.404839,0.090825,0.175747,0.166854,...,0.282179,-0.152177,0.219008,-0.175435,-0.491605,0.331040,0.248201,-0.064353,0.101061,0.182705
45008,-0.073689,0.370423,-0.106119,-0.007474,-0.050272,-0.202091,0.404839,0.090825,0.175747,0.166854,...,0.282179,-0.152177,0.219008,-0.175435,-0.491605,0.331040,0.248201,-0.064353,0.101061,0.182705
45009,-0.148996,0.454998,-0.168628,0.024398,-0.065070,-0.111850,0.150651,-0.066592,0.160821,-0.228624,...,0.111209,0.093540,0.125449,-0.237482,-0.396527,-0.079588,0.019276,-0.100971,0.022527,-0.063107


ответы

In [13]:
sentences = Ans
embeddings = model.encode(sentences)
len(embeddings)

45011

In [14]:
a_cols = []
for i in range (384):
    c = 'A' + str(i)
    a_cols.append(c)

In [15]:
A = pd.DataFrame(embeddings)
A.columns = a_cols
A

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A374,A375,A376,A377,A378,A379,A380,A381,A382,A383
0,-0.007696,0.598579,-0.215675,-0.007980,-0.015847,-0.188195,0.372676,0.062708,0.080712,0.105876,...,0.241284,0.078158,0.089408,-0.134405,-0.190672,0.187698,0.120142,0.070909,-0.124139,0.078369
1,-0.033416,0.398300,-0.162933,-0.078699,-0.123090,-0.154400,0.418404,0.262850,0.173844,-0.001274,...,0.192531,-0.137275,0.143819,-0.009310,-0.256419,0.088315,-0.024107,-0.103259,-0.162388,0.042545
2,0.074078,0.339487,-0.118172,-0.002772,0.061949,-0.236128,0.223978,0.223220,0.213070,-0.038284,...,0.335915,-0.006792,0.085738,0.057410,-0.410691,0.012290,0.212915,0.068644,-0.106049,0.005082
3,0.062535,0.216935,-0.057771,0.075541,-0.061746,-0.087207,0.269770,0.017559,0.223937,0.119476,...,0.147466,-0.129517,0.075499,0.063895,-0.175959,0.156325,0.243418,0.041970,-0.055779,0.032546
4,0.075382,0.273859,-0.131960,0.110961,0.081277,-0.147336,0.311410,0.111970,0.234782,0.080065,...,0.052048,-0.112282,0.138951,0.097625,-0.194535,0.174504,0.281476,-0.045258,0.031725,-0.027676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,0.197586,0.179002,-0.064668,-0.076510,0.009533,-0.158047,0.530381,0.201358,0.329076,0.126222,...,0.319299,-0.239305,0.261075,-0.011606,-0.391328,0.319050,0.241843,0.072983,-0.218374,0.170124
45007,-0.031303,0.220848,-0.135926,0.112492,-0.009229,-0.145211,0.499825,0.117611,0.348271,0.159026,...,0.334650,-0.238796,0.302667,-0.151646,-0.504632,0.174595,0.148690,-0.000977,-0.109513,0.128658
45008,-0.078956,0.343654,-0.087421,-0.054236,0.102972,-0.052167,0.366398,0.121682,0.255904,0.158206,...,0.362172,-0.147330,0.135424,-0.070508,-0.457227,0.119259,0.128970,-0.063903,0.100987,0.271797
45009,-0.103779,0.383879,-0.117835,0.081125,-0.090569,-0.282138,0.178121,0.044816,0.149035,-0.099929,...,0.209888,-0.102486,0.064728,-0.024921,-0.389627,0.019345,0.020398,0.001474,-0.022396,0.022390


In [16]:
A_copy = A.copy()
A_copy

,A0,A1,A2,A3,A4,A5,A6,A7,A8,A9,...,A374,A375,A376,A377,A378,A379,A380,A381,A382,A383
0,-0.007696,0.598579,-0.215675,-0.007980,-0.015847,-0.188195,0.372676,0.062708,0.080712,0.105876,...,0.241284,0.078158,0.089408,-0.134405,-0.190672,0.187698,0.120142,0.070909,-0.124139,0.078369
1,-0.033416,0.398300,-0.162933,-0.078699,-0.123090,-0.154400,0.418404,0.262850,0.173844,-0.001274,...,0.192531,-0.137275,0.143819,-0.009310,-0.256419,0.088315,-0.024107,-0.103259,-0.162388,0.042545
2,0.074078,0.339487,-0.118172,-0.002772,0.061949,-0.236128,0.223978,0.223220,0.213070,-0.038284,...,0.335915,-0.006792,0.085738,0.057410,-0.410691,0.012290,0.212915,0.068644,-0.106049,0.005082
3,0.062535,0.216935,-0.057771,0.075541,-0.061746,-0.087207,0.269770,0.017559,0.223937,0.119476,...,0.147466,-0.129517,0.075499,0.063895,-0.175959,0.156325,0.243418,0.041970,-0.055779,0.032546
4,0.075382,0.273859,-0.131960,0.110961,0.081277,-0.147336,0.311410,0.111970,0.234782,0.080065,...,0.052048,-0.112282,0.138951,0.097625,-0.194535,0.174504,0.281476,-0.045258,0.031725,-0.027676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,0.197586,0.179002,-0.064668,-0.076510,0.009533,-0.158047,0.530381,0.201358,0.329076,0.126222,...,0.319299,-0.239305,0.261075,-0.011606,-0.391328,0.319050,0.241843,0.072983,-0.218374,0.170124
45007,-0.031303,0.220848,-0.135926,0.112492,-0.009229,-0.145211,0.499825,0.117611,0.348271,0.159026,...,0.334650,-0.238796,0.302667,-0.151646,-0.504632,0.174595,0.148690,-0.000977,-0.109513,0.128658
45008,-0.078956,0.343654,-0.087421,-0.054236,0.102972,-0.052167,0.366398,0.121682,0.255904,0.158206,...,0.362172,-0.147330,0.135424,-0.070508,-0.457227,0.119259,0.128970,-0.063903,0.100987,0.271797
45009,-0.103779,0.383879,-0.117835,0.081125,-0.090569,-0.282138,0.178121,0.044816,0.149035,-0.099929,...,0.209888,-0.102486,0.064728,-0.024921,-0.389627,0.019345,0.020398,0.001474,-0.022396,0.022390


In [17]:
a_cols

['A0',
 'A1',
 'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'A10',
 'A11',
 'A12',
 'A13',
 'A14',
 'A15',
 'A16',
 'A17',
 'A18',
 'A19',
 'A20',
 'A21',
 'A22',
 'A23',
 'A24',
 'A25',
 'A26',
 'A27',
 'A28',
 'A29',
 'A30',
 'A31',
 'A32',
 'A33',
 'A34',
 'A35',
 'A36',
 'A37',
 'A38',
 'A39',
 'A40',
 'A41',
 'A42',
 'A43',
 'A44',
 'A45',
 'A46',
 'A47',
 'A48',
 'A49',
 'A50',
 'A51',
 'A52',
 'A53',
 'A54',
 'A55',
 'A56',
 'A57',
 'A58',
 'A59',
 'A60',
 'A61',
 'A62',
 'A63',
 'A64',
 'A65',
 'A66',
 'A67',
 'A68',
 'A69',
 'A70',
 'A71',
 'A72',
 'A73',
 'A74',
 'A75',
 'A76',
 'A77',
 'A78',
 'A79',
 'A80',
 'A81',
 'A82',
 'A83',
 'A84',
 'A85',
 'A86',
 'A87',
 'A88',
 'A89',
 'A90',
 'A91',
 'A92',
 'A93',
 'A94',
 'A95',
 'A96',
 'A97',
 'A98',
 'A99',
 'A100',
 'A101',
 'A102',
 'A103',
 'A104',
 'A105',
 'A106',
 'A107',
 'A108',
 'A109',
 'A110',
 'A111',
 'A112',
 'A113',
 'A114',
 'A115',
 'A116',
 'A117',
 'A118',
 'A119',
 'A120',
 'A121',
 'A122',
 'A

In [18]:
sci_fi_data_res = pd.concat([data, Q, A], axis=1, join='inner')
sci_fi_data_res.head(5)

,question_num,link,tags,question_author,question_text,question_author_reputation,question_author_likes,answer_author_name,answer_text,answer_reputation,...,A374,A375,A376,A377,A378,A379,A380,A381,A382,A383
0,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,EricSSH,This is Nam-Ek\n\nNam-Ek was born on the dista...,1383,...,0.241284,0.078158,0.089408,-0.134405,-0.190672,0.187698,0.120142,0.070909,-0.124139,0.078369
1,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,Omegacron,"This could also be Tor-An, a large Kryptonian ...",633000,...,0.192531,-0.137275,0.143819,-0.009310,-0.256419,0.088315,-0.024107,-0.103259,-0.162388,0.042545
2,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,CHRISTOPHER A. DAVIS DAVIS,That was Sciencist NON....the mentor /ex-frien...,1,...,0.335915,-0.006792,0.085738,0.057410,-0.410691,0.012290,0.212915,0.068644,-0.106049,0.005082
3,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Valorum,I asked my good friend Elizabeth (Lizzie) Yian...,695000,...,0.147466,-0.129517,0.075499,0.063895,-0.175959,0.156325,0.243418,0.041970,-0.055779,0.032546
4,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Kurumi83,The scriptures on Princess Ahmanet's face (Sof...,31,...,0.052048,-0.112282,0.138951,0.097625,-0.194535,0.174504,0.281476,-0.045258,0.031725,-0.027676


In [19]:
cols = sci_fi_data_res.columns

In [20]:
pd.DataFrame(sci_fi_data_res).to_csv("sci_fi_data_res.csv", index=False, header=False)

# 3. Разберемся с тегами

In [56]:
data = pd.read_csv('sci_fi_data_res.csv', names=cols, header=None)
data

,question_num,link,tags,question_author,question_text,question_author_reputation,question_author_likes,answer_author_name,answer_text,answer_reputation,...,A374,A375,A376,A377,A378,A379,A380,A381,A382,A383
0,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,EricSSH,This is Nam-Ek\n\nNam-Ek was born on the dista...,1383,...,0.241284,0.078158,0.089408,-0.134405,-0.190672,0.187698,0.120142,0.070909,-0.124139,0.078369
1,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,Omegacron,"This could also be Tor-An, a large Kryptonian ...",633000,...,0.192531,-0.137275,0.143819,-0.009310,-0.256419,0.088315,-0.024107,-0.103259,-0.162388,0.042545
2,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,CHRISTOPHER A. DAVIS DAVIS,That was Sciencist NON....the mentor /ex-frien...,1,...,0.335915,-0.006792,0.085738,0.057410,-0.410691,0.012290,0.212915,0.068644,-0.106049,0.005082
3,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Valorum,I asked my good friend Elizabeth (Lizzie) Yian...,695000,...,0.147466,-0.129517,0.075499,0.063895,-0.175959,0.156325,0.243418,0.041970,-0.055779,0.032546
4,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Kurumi83,The scriptures on Princess Ahmanet's face (Sof...,31,...,0.052048,-0.112282,0.138951,0.097625,-0.194535,0.174504,0.281476,-0.045258,0.031725,-0.027676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,The Spooniest,"At least as of the first movie, Obi-Wan can't ...",3166,...,0.319299,-0.239305,0.261075,-0.011606,-0.391328,0.319050,0.241843,0.072983,-0.218374,0.170124
45007,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Peter Cordes,"Obi-Wan has the power to persuade minds, so wh...",525,...,0.334650,-0.238796,0.302667,-0.151646,-0.504632,0.174595,0.148690,-0.000977,-0.109513,0.128658
45008,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Tim,Want to improve this answer? Add details and i...,6237,...,0.362172,-0.147330,0.135424,-0.070508,-0.457227,0.119259,0.128970,-0.063903,0.100987,0.271797
45009,17999,https://scifi.stackexchange.com/questions/1342...,"['harry-potter', 'voldemort']",Slytherincess,Is there verification either in canon or from ...,165000,5,DVK-on-Ahch-To,Christiana: How did voldemort get his wand bac...,343000,...,0.209888,-0.102486,0.064728,-0.024921,-0.389627,0.019345,0.020398,0.001474,-0.022396,0.022390


In [20]:
data = pd.read_csv('sci_fi_data_res.csv', header=None)
data = data.rename(columns={0 : 'question_num',  1 : 'link', 2 : 'tags', 3 : 'question_author', 4 :	'question_text',	5 : 'question_author_reputation', 6 : 'question_author_likes',
                            7 : 'answer_author_name', 8 :	'answer_text', 9 : 'answer_reputation', 10 : 'answer_likes', 11 : 'num_answer'})
data

,question_num,link,tags,question_author,question_text,question_author_reputation,question_author_likes,answer_author_name,answer_text,answer_reputation,...,770,771,772,773,774,775,776,777,778,779
0,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,EricSSH,This is Nam-Ek\n\nNam-Ek was born on the dista...,1383,...,0.241284,0.078158,0.089408,-0.134405,-0.190672,0.187698,0.120142,0.070909,-0.124139,0.078369
1,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,Omegacron,"This could also be Tor-An, a large Kryptonian ...",633000,...,0.192531,-0.137275,0.143819,-0.009310,-0.256419,0.088315,-0.024107,-0.103259,-0.162388,0.042545
2,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,CHRISTOPHER A. DAVIS DAVIS,That was Sciencist NON....the mentor /ex-frien...,1,...,0.335915,-0.006792,0.085738,0.057410,-0.410691,0.012290,0.212915,0.068644,-0.106049,0.005082
3,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Valorum,I asked my good friend Elizabeth (Lizzie) Yian...,695000,...,0.147466,-0.129517,0.075499,0.063895,-0.175959,0.156325,0.243418,0.041970,-0.055779,0.032546
4,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Kurumi83,The scriptures on Princess Ahmanet's face (Sof...,31,...,0.052048,-0.112282,0.138951,0.097625,-0.194535,0.174504,0.281476,-0.045258,0.031725,-0.027676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,The Spooniest,"At least as of the first movie, Obi-Wan can't ...",3166,...,0.319299,-0.239305,0.261075,-0.011606,-0.391328,0.319050,0.241843,0.072983,-0.218374,0.170124
45007,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Peter Cordes,"Obi-Wan has the power to persuade minds, so wh...",525,...,0.334650,-0.238796,0.302667,-0.151646,-0.504632,0.174595,0.148690,-0.000977,-0.109513,0.128658
45008,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Tim,Want to improve this answer? Add details and i...,6237,...,0.362172,-0.147330,0.135424,-0.070508,-0.457227,0.119259,0.128970,-0.063903,0.100987,0.271797
45009,17999,https://scifi.stackexchange.com/questions/1342...,"['harry-potter', 'voldemort']",Slytherincess,Is there verification either in canon or from ...,165000,5,DVK-on-Ahch-To,Christiana: How did voldemort get his wand bac...,343000,...,0.209888,-0.102486,0.064728,-0.024921,-0.389627,0.019345,0.020398,0.001474,-0.022396,0.022390


In [21]:
data['answer_author_name'].value_counts()[:100]

answer_author_name
Valorum            2654
FuzzyBoots         1654
DVK-on-Ahch-To     1376
Thaddeus Howze     1026
TheLethalCarrot     779
                   ... 
Jeremy French        36
Radhil               36
Zeiss Ikon           35
rs.29                35
Stark07              35
Name: count, Length: 100, dtype: int64

разберемся с тегами, сделаем аналог OHE для топ 100

In [22]:
tags = data['tags']
len(tags)

45011

In [23]:
_tags = []
import re
for tag in tags:
    tag = tag.replace("'", '')
    tag = tag.replace('[', '')
    tag = tag.replace(']', '')
    tag = tag.replace(',', '')
    _tags.append(re.split(" ", tag))

In [24]:
list_tags = []
for tag in _tags:
  for t in tag:
    if t != '':  # убираем безтеговые вопросы
        list_tags.append(t)
len(list_tags)

103770

In [25]:
df_list_tags = pd.DataFrame(list_tags)
df_list_tags = df_list_tags[0].value_counts().head(100)
#df_list_tags = df_list_tags[0].value_counts().head(10)
lt = pd.DataFrame(df_list_tags)

In [26]:
feature_list = lt.index
tags_df = pd.DataFrame(0, index=np.arange(45011), columns=feature_list)
tags_df

,harry-potter,star-wars,story-identification,star-trek,marvel,tolkiens-legendarium,the-lord-of-the-rings,marvel-cinematic-universe,dc,star-trek-tng,...,the-wheel-of-time,horror,iron-man,sorting-hat,zombie,enders-game,the-one-ring,anime,character-motivation,adaptation-comparison
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45007,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
45009,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
list_tags = list(_tags)
for i in range (45011):
    for col in tags_df.columns:
        tags_df[col][i] = list_tags[i].count(col)

In [28]:
data_res = pd.concat([data, tags_df], axis=1, join='inner')
data_res

,question_num,link,tags,question_author,question_text,question_author_reputation,question_author_likes,answer_author_name,answer_text,answer_reputation,...,the-wheel-of-time,horror,iron-man,sorting-hat,zombie,enders-game,the-one-ring,anime,character-motivation,adaptation-comparison
0,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,EricSSH,This is Nam-Ek\n\nNam-Ek was born on the dista...,1383,...,0,0,0,0,0,0,0,0,0,0
1,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,Omegacron,"This could also be Tor-An, a large Kryptonian ...",633000,...,0,0,0,0,0,0,0,0,0,0
2,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,CHRISTOPHER A. DAVIS DAVIS,That was Sciencist NON....the mentor /ex-frien...,1,...,0,0,0,0,0,0,0,0,0,0
3,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Valorum,I asked my good friend Elizabeth (Lizzie) Yian...,695000,...,0,0,0,0,0,0,0,0,0,0
4,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Kurumi83,The scriptures on Princess Ahmanet's face (Sof...,31,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,The Spooniest,"At least as of the first movie, Obi-Wan can't ...",3166,...,0,0,0,0,0,0,0,0,0,0
45007,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Peter Cordes,"Obi-Wan has the power to persuade minds, so wh...",525,...,0,0,0,0,0,0,0,0,0,0
45008,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Tim,Want to improve this answer? Add details and i...,6237,...,0,0,0,0,0,0,0,0,0,0
45009,17999,https://scifi.stackexchange.com/questions/1342...,"['harry-potter', 'voldemort']",Slytherincess,Is there verification either in canon or from ...,165000,5,DVK-on-Ahch-To,Christiana: How did voldemort get his wand bac...,343000,...,0,0,0,0,0,0,0,0,0,0


In [29]:
pd.DataFrame(data_res).to_csv("data_res.csv", index=False, header=False)

# 4. ML models

In [2]:
data = pd.read_csv('data_res.csv', header=None)
#question_num	link	tags	question_author	question_text	question_author_reputation	question_author_likes	answer_author_name	answer_text	answer_reputation
data = data.rename(columns={0 : 'question_num',  1 : 'link', 2 : 'tags', 3 : 'question_author', 4 :	'question_text',	5 : 'question_author_reputation', 6 : 'question_author_likes',
                            7 : 'answer_author_name', 8 :	'answer_text', 9 : 'answer_reputation', 10 : 'answer_likes', 11 : 'num_answer'})
data = data.rename(str,axis="columns")
data

,question_num,link,tags,question_author,question_text,question_author_reputation,question_author_likes,answer_author_name,answer_text,answer_reputation,...,870,871,872,873,874,875,876,877,878,879
0,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,EricSSH,This is Nam-Ek\n\nNam-Ek was born on the dista...,1383,...,0,0,0,0,0,0,0,0,0,0
1,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,Omegacron,"This could also be Tor-An, a large Kryptonian ...",633000,...,0,0,0,0,0,0,0,0,0,0
2,1,https://scifi.stackexchange.com/questions/6887...,"['dc', 'superman', 'character-identification',...",Kreann,"In Man of Steel, when Zod attacks the Council,...",8407,13,CHRISTOPHER A. DAVIS DAVIS,That was Sciencist NON....the mentor /ex-frien...,1,...,0,0,0,0,0,0,0,0,0,0
3,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Valorum,I asked my good friend Elizabeth (Lizzie) Yian...,695000,...,0,0,0,0,0,0,0,0,0,0
4,2,https://scifi.stackexchange.com/questions/1614...,['mummy-2017'],CodeMed,"In The Mummy 2017, the title character has sym...",108000,12,Kurumi83,The scriptures on Princess Ahmanet's face (Sof...,31,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,The Spooniest,"At least as of the first movie, Obi-Wan can't ...",3166,...,0,0,0,0,0,0,0,0,0,0
45007,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Peter Cordes,"Obi-Wan has the power to persuade minds, so wh...",525,...,0,0,0,0,0,0,0,0,0,0
45008,17998,https://scifi.stackexchange.com/questions/1412...,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",King of NES,"These aren't the droids you're looking for, yo...",5466,48,Tim,Want to improve this answer? Add details and i...,6237,...,0,0,0,0,0,0,0,0,0,0
45009,17999,https://scifi.stackexchange.com/questions/1342...,"['harry-potter', 'voldemort']",Slytherincess,Is there verification either in canon or from ...,165000,5,DVK-on-Ahch-To,Christiana: How did voldemort get his wand bac...,343000,...,0,0,0,0,0,0,0,0,0,0


In [6]:
'''
data = data_res
data = data.rename(str,axis="columns")
data
'''

'\ndata = data_res\ndata = data.rename(str,axis="columns")\ndata\n'

In [3]:
data = data.drop(columns=['link', 'question_text', 'answer_text'])

In [4]:
data = data.drop(columns=['question_author', 'answer_author_name', 'question_author_likes'])
#data = data.drop(columns=['question_author_likes'])
#data['answer_author_name'] = data['answer_author_name'].astype('category')
#data['question_author'] = data['question_author'].astype('category')

In [5]:
data['num_answer'] = data['num_answer'].apply(lambda x: 0 if x > 1 else x)
data.head(5)

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,870,871,872,873,874,875,876,877,878,879
0,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
1,1,"['dc', 'superman', 'character-identification',...",8407,633000,4,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
2,1,"['dc', 'superman', 'character-identification',...",8407,1,-1,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
3,2,['mummy-2017'],108000,695000,21,1,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
4,2,['mummy-2017'],108000,31,3,0,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0


In [6]:
from scipy import stats
#data[np.abs(stats.zscore(data['answer_reputation'])) > 2]
d = data[np.abs(stats.zscore(data['question_author_reputation'])) > 2]
qa_rep_tr = d['question_author_reputation'].min()

In [7]:
from scipy import stats
#data[np.abs(stats.zscore(data['answer_reputation'])) > 2]
d = data[np.abs(stats.zscore(data['answer_reputation'])) > 2]
ans_rep_tr = d['answer_reputation'].min()

In [8]:
from scipy import stats
#data[np.abs(stats.zscore(data['answer_reputation'])) > 2]
d = data[np.abs(stats.zscore(data['answer_likes'])) > 2]
likes_tr = d['answer_likes'].min()

In [9]:
print(qa_rep_tr, ans_rep_tr, likes_tr)

569000 774000 48


In [10]:
data['question_author_reputation'] = data['question_author_reputation'].apply (lambda x: x if x < qa_rep_tr else qa_rep_tr)
data['answer_reputation'] = data['answer_reputation'].apply (lambda x: x if x < ans_rep_tr else ans_rep_tr)
data['answer_likes'] = data['answer_likes'].apply (lambda x: x if x < likes_tr else likes_tr)
data

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,870,871,872,873,874,875,876,877,878,879
0,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
1,1,"['dc', 'superman', 'character-identification',...",8407,633000,4,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
2,1,"['dc', 'superman', 'character-identification',...",8407,1,-1,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
3,2,['mummy-2017'],108000,695000,21,1,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
4,2,['mummy-2017'],108000,31,3,0,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,3166,8,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45007,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,525,3,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45008,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,6237,1,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45009,17999,"['harry-potter', 'voldemort']",165000,343000,2,1,-0.148996,0.454998,-0.168628,0.024398,...,0,0,0,0,0,0,0,0,0,0


In [15]:
data_train = data[data['question_num'] < 15000]
data_test = data[data['question_num'] >= 15000]

In [16]:
X_train, X_test = data_train.drop(columns=['tags', 'answer_likes', 'num_answer', 'question_num']), data_test.drop(columns=['tags', 'answer_likes', 'num_answer', 'question_num'])
y_train, y_test = data_train['num_answer'], data_test['num_answer']

In [17]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(37477, 870) (7534, 870)
(37477,) (7534,)


In [18]:
X_train.dtypes

question_author_reputation      int64
answer_reputation               int64
12                            float64
13                            float64
14                            float64
                               ...   
875                             int64
876                             int64
877                             int64
878                             int64
879                             int64
Length: 870, dtype: object

## 3.1 Logistic regression

In [69]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [66]:
LogReg = LogisticRegression(solver='lbfgs', tol=0.0001, C=1.0, max_iter=20000)
LogReg.fit(X_train, y_train)

LogisticRegression(max_iter=20000)

In [67]:
y_pred_lr = LogReg.predict(X_test)

print(f'Logistic Regression F1 Score {f1_score(y_test, y_pred_lr)}')
print(f'Logistic Regression Accuracy {accuracy_score(y_test, y_pred_lr)}')

Logistic Regression F1 Score 0.6143049126474541
Logistic Regression Accuracy 0.6571542341385718


In [68]:
cm_lr = pd.DataFrame(confusion_matrix(y_test, y_pred_lr))
cm_lr

,0,1
0,2894,1687
1,896,2057


In [70]:
specificity = cm_lr[1][1] / (cm_lr[0][1] + cm_lr[1][1])

print('Доля верно обнаруженных лучших : ', specificity)

Доля верно обнаруженных лучших :  0.6965797494073823


In [71]:
LR_metric = [accuracy_score(y_test, y_pred_lr), f1_score(y_test, y_pred_lr), specificity]
LR_metric

[0.6571542341385718, 0.6143049126474541, 0.6965797494073823]

## 3.2 RandomForestClassifier

In [72]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()
RF.fit(X_train, y_train)

RandomForestClassifier()

In [73]:
y_pred_rf = RF.predict(X_test)

print(f'Random forest F1 Score {f1_score(y_test, y_pred_rf)}')
print(f'Random forest Accuracy {accuracy_score(y_test, y_pred_rf)}')

Random forest F1 Score 0.6602171767028627
Random forest Accuracy 0.7715688877090523


In [74]:
from sklearn.metrics import confusion_matrix

cm_rf = pd.DataFrame(confusion_matrix(y_test, y_pred_rf))
cm_rf

,0,1
0,4141,440
1,1281,1672


In [75]:
specificity = cm_rf[1][1] / (cm_rf[0][1] + cm_rf[1][1])

print('Доля верно обнаруженных лучших : ', specificity)

Доля верно обнаруженных лучших :  0.5662038604808669


In [76]:
RF_metric = [accuracy_score(y_test, y_pred_rf), f1_score(y_test, y_pred_rf), specificity]
RF_metric

[0.7715688877090523, 0.6602171767028627, 0.5662038604808669]

## 3.3 CatBoostClassifier

In [77]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.3 MB/s eta 0:00:00


In [ ]:
#from catboost import CatBoostClassifier

#cat_features = ['question_num',	'question_author', 'answer_author_name']  # List of categorical features

#CatBoost = CatBoostClassifier(verbose=False)
#CatBoost = CatBoostClassifier(verbose=False, cat_features=cat_features)
#CatBoost.fit(X_train, y_train)

In [78]:
from catboost import CatBoostClassifier

CatBoost = CatBoostClassifier(verbose=False)
CatBoost.fit(X_train, y_train)

In [79]:
y_pred_cb = CatBoost.predict(X_test)

print(f'CatBoost F1 Score {f1_score(y_test, y_pred_cb)}')
print(f'CatBoost Accuracy {accuracy_score(y_test, y_pred_cb)}')

CatBoost F1 Score 0.7548344804981973
CatBoost Accuracy 0.8014335014600478


In [80]:
cm_cb = pd.DataFrame(confusion_matrix(y_test, y_pred_cb))
cm_cb

,0,1
0,3735,846
1,650,2303


In [81]:
specificity = cm_cb[1][1] / (cm_cb[0][1] + cm_cb[1][1])

print('Доля верно обнаруженных лучших : ', specificity)

Доля верно обнаруженных лучших :  0.7798848628513376


In [82]:
CB_metric = [accuracy_score(y_test, y_pred_cb), f1_score(y_test, y_pred_cb), specificity]
CB_metric

[0.8014335014600478, 0.7548344804981973, 0.7798848628513376]

## 3.4 Результаты

In [83]:
res = pd.DataFrame()

res['LR'] = LR_metric
res['RF'] = RF_metric
res['CB'] = CB_metric

res.index = ['Точность', 'F1 мера', 'Доля верно обнаруженных лучших ответов']
res = res.round(3)
res

,LR,RF,CB
Точность,0.657,0.772,0.801
F1 мера,0.614,0.660,0.755
Доля верно обнаруженных лучших ответов,0.697,0.566,0.780


# 4. Попробуем сделать мультикласс


In [152]:
data = data_res

In [153]:
data = data.drop(columns=['link', 'question_text', 'answer_text'])

In [154]:
data = data.drop(columns=['question_author', 'answer_author_name', 'question_author_likes'])

In [155]:
data['num_answer'] = data['num_answer'].apply(lambda x: 0 if x > 2 else x)
data.head(5)

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,the-wheel-of-time,horror,iron-man,sorting-hat,zombie,enders-game,the-one-ring,anime,character-motivation,adaptation-comparison
0,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
1,1,"['dc', 'superman', 'character-identification',...",8407,633000,4,2,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
2,1,"['dc', 'superman', 'character-identification',...",8407,1,-1,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
3,2,['mummy-2017'],108000,695000,21,1,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
4,2,['mummy-2017'],108000,31,3,2,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0


In [156]:
data_train = data[data['question_num'] < 15000]
data_test = data[data['question_num'] >= 15000]

In [157]:
X_train, X_test = data_train.drop(columns=['tags', 'answer_likes', 'num_answer']), data_test.drop(columns=['tags', 'answer_likes', 'num_answer'])
y_train, y_test = data_train['num_answer'], data_test['num_answer']

In [158]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(37477, 871) (7534, 871)
(37477,) (7534,)


## 4.1 CatBoostClassifier

In [159]:
from catboost import CatBoostClassifier

CatBoost = CatBoostClassifier(verbose=False)
CatBoost.fit(X_train, y_train)

In [160]:
y_pred_cb = CatBoost.predict(X_test)

#print(f'CatBoost F1 Score {f1_score(y_test, y_pred_cb)}')
print(f'CatBoost Accuracy {accuracy_score(y_test, y_pred_cb)}')

CatBoost Accuracy 0.6748075391558269


In [161]:
cm_cb = pd.DataFrame(confusion_matrix(y_test, y_pred_cb))
cm_cb

,0,1,2
0,2125,544,134
1,228,2631,94
2,562,888,328


In [162]:
y_cb = pd.DataFrame(y_pred_cb)
y_cb[0] = y_cb[0].apply(lambda x: 0 if x == 0 else 1)
y_pred_cb = y_cb[0]

In [163]:
y_t = pd.DataFrame(y_test)
y_t

,num_answer
37477,1
37478,2
37479,1
37480,1
37481,1
...,...
45006,0
45007,0
45008,0
45009,1


In [164]:
y_t = pd.DataFrame(y_test)
y_t[0] = y_t['num_answer'].apply(lambda x: 0 if x == 0 else 1)
y_test = y_t[0]

In [165]:
print(f'CatBoost F1 Score {f1_score(y_test, y_pred_cb)}')
print(f'CatBoost Accuracy {accuracy_score(y_test, y_pred_cb)}')

CatBoost F1 Score 0.8429946524064171
CatBoost Accuracy 0.8051499867268384


In [170]:
cm_cb = pd.DataFrame(confusion_matrix(y_test, y_pred_cb))
cm_cb

,0,1
0,2125,678
1,790,3941


# 5. Попробуем регрессию

In [11]:
data = data_res

NameError: name 'data_res' is not defined

In [ ]:
data = data.drop(columns=['link', 'question_text', 'answer_text'])

In [31]:
data = data.drop(columns=['question_author', 'answer_author_name', 'question_author_likes'])

In [32]:
data['num_answer'] = data['num_answer'].apply(lambda x: 0 if x > 1 else x)
data.head(5)

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,870,871,872,873,874,875,876,877,878,879
0,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
1,1,"['dc', 'superman', 'character-identification',...",8407,633000,4,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
2,1,"['dc', 'superman', 'character-identification',...",8407,1,-1,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
3,2,['mummy-2017'],108000,695000,21,1,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
4,2,['mummy-2017'],108000,31,3,0,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0


In [13]:
data

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,870,871,872,873,874,875,876,877,878,879
0,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
1,1,"['dc', 'superman', 'character-identification',...",8407,633000,4,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
2,1,"['dc', 'superman', 'character-identification',...",8407,1,-1,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
3,2,['mummy-2017'],108000,695000,21,1,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
4,2,['mummy-2017'],108000,31,3,0,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,3166,8,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45007,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,525,3,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45008,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,6237,1,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45009,17999,"['harry-potter', 'voldemort']",165000,343000,2,1,-0.148996,0.454998,-0.168628,0.024398,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data_train = data[data['question_num'] < 15000]
data_test = data[data['question_num'] >= 15000]

In [15]:
X_train, X_test = data_train.drop(columns=['tags', 'answer_likes', 'num_answer', 'question_num']), data_test.drop(columns=['tags', 'answer_likes', 'num_answer', 'question_num'])
y_train, y_test = data_train['answer_likes'], data_test['answer_likes']

In [16]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(37477, 870) (7534, 870)
(37477,) (7534,)


Catboost

In [92]:
from catboost import CatBoostRegressor

#regressor = CatBoostRegressor(iterations=200, depth=4, learning_rate=0.01)
regressor = CatBoostRegressor()
regressor.fit(X_train, y_train, verbose=False)

In [93]:
# Предсказание и оценка
pred = regressor.predict(X_test)
print(pred)

[ 6.28406791  1.40124307 19.83617687 ... 15.00782761 10.46102538
 15.37894787]


In [94]:
len(pred)

7534

In [95]:
predictions = pd.DataFrame()
predictions['question_num'] = data_test['question_num']
predictions['answer_likes'] = data_test['answer_likes']
predictions['num_answer'] = data_test['num_answer']
predictions['answer_likes_pred'] = pred
predictions

,question_num,answer_likes,num_answer,answer_likes_pred
37477,15000,5,1,6.284068
37478,15000,2,0,1.401243
37479,15001,17,1,19.836177
37480,15002,6,1,11.402049
37481,15003,9,1,11.497710
...,...,...,...,...
45006,17998,8,0,10.940766
45007,17998,3,0,9.454409
45008,17998,1,0,15.007828
45009,17999,2,1,10.461025


In [96]:
sorted_pred = predictions.sort_values(by = ['question_num', 'answer_likes_pred'], ascending = [True, False])
sorted_pred

,question_num,answer_likes,num_answer,answer_likes_pred
37477,15000,5,1,6.284068
37478,15000,2,0,1.401243
37479,15001,17,1,19.836177
37480,15002,6,1,11.402049
37481,15003,9,1,11.497710
...,...,...,...,...
45005,17998,30,0,11.394307
45006,17998,8,0,10.940766
45007,17998,3,0,9.454409
45009,17999,2,1,10.461025


In [97]:
sorted_pred['num_answer_pred'] = 0
sorted_pred.reset_index(drop=True, inplace=True)
sorted_pred

,question_num,answer_likes,num_answer,answer_likes_pred,num_answer_pred
0,15000,5,1,6.284068,0
1,15000,2,0,1.401243,0
2,15001,17,1,19.836177,0
3,15002,6,1,11.402049,0
4,15003,9,1,11.497710,0
...,...,...,...,...,...
7529,17998,30,0,11.394307,0
7530,17998,8,0,10.940766,0
7531,17998,3,0,9.454409,0
7532,17999,2,1,10.461025,0


In [ ]:
q_num = 0
for i in range (7534):
    if sorted_pred['question_num'][i] != q_num:
        q_num = sorted_pred['question_num'][i]
        sorted_pred['num_answer_pred'][i] = 1

In [99]:
sorted_pred

,question_num,answer_likes,num_answer,answer_likes_pred,num_answer_pred
0,15000,5,1,6.284068,1
1,15000,2,0,1.401243,0
2,15001,17,1,19.836177,1
3,15002,6,1,11.402049,1
4,15003,9,1,11.497710,1
...,...,...,...,...,...
7529,17998,30,0,11.394307,0
7530,17998,8,0,10.940766,0
7531,17998,3,0,9.454409,0
7532,17999,2,1,10.461025,1


In [100]:
y_pred = sorted_pred['num_answer_pred']
y_test = sorted_pred['num_answer']

print(f'CatBoost F1 Score {f1_score(y_test, y_pred)}')
print(f'CatBoost Accuracy {accuracy_score(y_test, y_pred)}')

CatBoost F1 Score 0.7636302065695902
CatBoost Accuracy 0.8147066631271569


In [101]:
cm_cb = pd.DataFrame(confusion_matrix(y_test, y_pred))
cm_cb

,0,1
0,3883,698
1,698,2255


In [102]:
specificity = cm_cb[1][1] / (cm_cb[0][1] + cm_cb[1][1])

print('Доля верно обнаруженных лучших : ', specificity)

Доля верно обнаруженных лучших :  0.7636302065695902


In [106]:
CB_reg_metric = [accuracy_score(y_test, y_pred_cb), f1_score(y_test, y_pred_cb), specificity]
CB_reg_metric

[0.8224953543934165, 0.7761455260570306, 0.7836302065695901]

In [107]:
res = pd.DataFrame()

res['CB_classification'] = [0.801, 0.755, 0.780]
res['CB_regression'] = CB_reg_metric

res.index = ['Точность', 'F1 мера', 'Доля верно обнаруженных лучших ответов']
res = res.round(3)
res

,CB_classification,CB_regression
Точность,0.801,0.822
F1 мера,0.755,0.776
Доля верно обнаруженных лучших ответов,0.780,0.784


# 7. Предсказываем без ответа

In [17]:
data = data.rename(str,axis="columns")
data

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,870,871,872,873,874,875,876,877,878,879
0,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
1,1,"['dc', 'superman', 'character-identification',...",8407,633000,4,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
2,1,"['dc', 'superman', 'character-identification',...",8407,1,-1,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
3,2,['mummy-2017'],108000,695000,21,1,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
4,2,['mummy-2017'],108000,31,3,0,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,3166,8,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45007,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,525,3,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45008,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,6237,1,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45009,17999,"['harry-potter', 'voldemort']",165000,343000,2,1,-0.148996,0.454998,-0.168628,0.024398,...,0,0,0,0,0,0,0,0,0,0


In [18]:
col = []
for i in range (780 - 384, 780):
    col.append(str(i))
data_mod = data.drop(columns=col)
data_mod

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,870,871,872,873,874,875,876,877,878,879
0,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
1,1,"['dc', 'superman', 'character-identification',...",8407,633000,4,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
2,1,"['dc', 'superman', 'character-identification',...",8407,1,-1,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
3,2,['mummy-2017'],108000,695000,21,1,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
4,2,['mummy-2017'],108000,31,3,0,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,3166,8,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45007,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,525,3,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45008,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,6237,1,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45009,17999,"['harry-potter', 'voldemort']",165000,343000,2,1,-0.148996,0.454998,-0.168628,0.024398,...,0,0,0,0,0,0,0,0,0,0


In [19]:
data_train = data_mod[data_mod['question_num'] < 15000]
data_test = data_mod[data_mod['question_num'] >= 15000]

In [20]:
X_train, X_test = data_train.drop(columns=['tags', 'answer_likes', 'num_answer', 'question_num']), data_test.drop(columns=['tags', 'answer_likes', 'num_answer', 'question_num'])
#y_train, y_test = data_train['answer_likes'], data_test['answer_likes']
y_train, y_test = data_train['num_answer'], data_test['num_answer']

In [21]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(37477, 486) (7534, 486)
(37477,) (7534,)


In [22]:
X_train

,question_author_reputation,answer_reputation,12,13,14,15,16,17,18,19,...,870,871,872,873,874,875,876,877,878,879
0,8407,1383,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,...,0,0,0,0,0,0,0,0,0,0
1,8407,633000,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,...,0,0,0,0,0,0,0,0,0,0
2,8407,1,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,...,0,0,0,0,0,0,0,0,0,0
3,108000,695000,0.003884,0.167487,-0.091520,0.130714,-0.096572,-0.141100,0.386765,-0.116209,...,0,0,0,0,0,0,0,0,0,0
4,108000,31,0.003884,0.167487,-0.091520,0.130714,-0.096572,-0.141100,0.386765,-0.116209,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37472,331000,104000,0.038582,0.309145,-0.267616,0.263565,-0.045215,-0.003271,0.224649,0.020339,...,0,0,0,0,0,0,0,0,0,0
37473,331000,605000,0.038582,0.309145,-0.267616,0.263565,-0.045215,-0.003271,0.224649,0.020339,...,0,0,0,0,0,0,0,0,0,0
37474,1614,165000,-0.121199,0.311127,-0.234067,0.069905,0.054047,-0.190292,0.421074,0.094105,...,0,0,0,0,0,0,0,0,0,0
37475,1614,41,-0.121199,0.311127,-0.234067,0.069905,0.054047,-0.190292,0.421074,0.094105,...,0,0,0,0,0,0,0,0,0,0


RandomForestClassifier

In [23]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()
RF.fit(X_train, y_train)

RandomForestClassifier()

In [77]:
y_pred_rf = RF.predict(X_test)

print(f'Random forest F1 Score {f1_score(y_test, y_pred_rf)}')
print(f'Random forest Accuracy {accuracy_score(y_test, y_pred_rf)}')

Random forest F1 Score 0.764327485380117
Random forest Accuracy 0.7860366339262013


In [78]:
from sklearn.metrics import confusion_matrix

cm_rf = pd.DataFrame(confusion_matrix(y_test, y_pred_rf))
cm_rf

,0,1
0,3308,1273
1,339,2614


In [79]:
specificity = cm_rf[1][1] / (cm_rf[0][1] + cm_rf[1][1])

print('Доля верно обнаруженных лучших : ', specificity)

Доля верно обнаруженных лучших :  0.8852014900101591


In [80]:
RF_metric = [accuracy_score(y_test, y_pred_rf), f1_score(y_test, y_pred_rf), specificity]
RF_metric

[0.7860366339262013, 0.764327485380117, 0.8852014900101591]

CatBoostRegressor

In [24]:
from catboost import CatBoostRegressor

#regressor = CatBoostRegressor(iterations=200, depth=4, learning_rate=0.01)
regressor = CatBoostRegressor()
regressor.fit(X_train, y_train, verbose=False)

In [61]:
# Предсказание и оценка
pred = regressor.predict(X_test)
print(pred)

[ 5.33408917  1.19622592 17.44860305 ... 11.82936645 11.10896442
 15.9113657 ]


In [62]:
len(pred)

7534

In [63]:
predictions = pd.DataFrame()
predictions['question_num'] = data_test['question_num']
predictions['answer_likes'] = data_test['answer_likes']
predictions['num_answer'] = data_test['num_answer']
predictions['answer_likes_pred'] = pred
predictions

,question_num,answer_likes,num_answer,answer_likes_pred
37477,15000,5,1,5.334089
37478,15000,2,0,1.196226
37479,15001,17,1,17.448603
37480,15002,6,1,8.013917
37481,15003,9,1,12.418562
...,...,...,...,...
45006,17998,8,0,9.808052
45007,17998,3,0,7.343200
45008,17998,1,0,11.829366
45009,17999,2,1,11.108964


In [64]:
sorted_pred = predictions.sort_values(by = ['question_num', 'answer_likes_pred'], ascending = [True, False])
sorted_pred

,question_num,answer_likes,num_answer,answer_likes_pred
37477,15000,5,1,5.334089
37478,15000,2,0,1.196226
37479,15001,17,1,17.448603
37480,15002,6,1,8.013917
37481,15003,9,1,12.418562
...,...,...,...,...
45006,17998,8,0,9.808052
45007,17998,3,0,7.343200
45005,17998,30,0,7.094802
45009,17999,2,1,11.108964


In [65]:
sorted_pred['num_answer_pred'] = 0
sorted_pred.reset_index(drop=True, inplace=True)
sorted_pred

,question_num,answer_likes,num_answer,answer_likes_pred,num_answer_pred
0,15000,5,1,5.334089,0
1,15000,2,0,1.196226,0
2,15001,17,1,17.448603,0
3,15002,6,1,8.013917,0
4,15003,9,1,12.418562,0
...,...,...,...,...,...
7529,17998,8,0,9.808052,0
7530,17998,3,0,7.343200,0
7531,17998,30,0,7.094802,0
7532,17999,2,1,11.108964,0


In [ ]:
q_num = 0
for i in range (7534):
    if sorted_pred['question_num'][i] != q_num:
        q_num = sorted_pred['question_num'][i]
        sorted_pred['num_answer_pred'][i] = 1

In [67]:
sorted_pred

,question_num,answer_likes,num_answer,answer_likes_pred,num_answer_pred
0,15000,5,1,5.334089,1
1,15000,2,0,1.196226,0
2,15001,17,1,17.448603,1
3,15002,6,1,8.013917,1
4,15003,9,1,12.418562,1
...,...,...,...,...,...
7529,17998,8,0,9.808052,0
7530,17998,3,0,7.343200,0
7531,17998,30,0,7.094802,0
7532,17999,2,1,11.108964,1


In [83]:
y_pred = sorted_pred['num_answer_pred']
y_test = sorted_pred['num_answer']

print(f'CatBoost F1 Score {f1_score(y_test, y_pred)}')
print(f'CatBoost Accuracy {accuracy_score(y_test, y_pred)}')

CatBoost F1 Score 0.758550626481544
CatBoost Accuracy 0.8107247146270241


In [84]:
cm_cb = pd.DataFrame(confusion_matrix(y_test, y_pred))
cm_cb

,0,1
0,3868,713
1,713,2240


In [85]:
specificity = cm_cb[1][1] / (cm_cb[0][1] + cm_cb[1][1])

print('Доля верно обнаруженных лучших : ', specificity)

Доля верно обнаруженных лучших :  0.7585506264815441


In [87]:
CB_metric = [accuracy_score(y_test, y_pred), f1_score(y_test, y_pred), specificity]
CB_metric

[0.8107247146270241, 0.758550626481544, 0.7585506264815441]

In [92]:
res = pd.DataFrame()

res['CB_classification'] = [0.801, 0.755, 0.780]
res['CB_regression'] = [0.8224953543934165, 0.7761455260570306, 0.7836302065695901]
res['RF_without_ans_txt'] = RF_metric
res['CB_without_ans_txt'] = CB_metric

res.index = ['Точность', 'F1 мера', 'Доля верно обнаруженных лучших ответов']
res = res.round(3)
res

,CB_classification,CB_regression,RF_without_ans_txt,CB_without_ans_txt
Точность,0.801,0.822,0.786,0.811
F1 мера,0.755,0.776,0.764,0.759
Доля верно обнаруженных лучших ответов,0.780,0.784,0.885,0.759


# Предсказываем рекомендованных

In [117]:
data_mod

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,870,871,872,873,874,875,876,877,878,879
0,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
1,1,"['dc', 'superman', 'character-identification',...",8407,633000,4,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
2,1,"['dc', 'superman', 'character-identification',...",8407,1,-1,0,-0.215233,0.347350,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
3,2,['mummy-2017'],108000,695000,21,1,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
4,2,['mummy-2017'],108000,31,3,0,0.003884,0.167487,-0.091520,0.130714,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45006,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,3166,8,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45007,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,525,3,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45008,17998,"['star-wars', 'the-force', 'a-new-hope', 'obi-...",5466,6237,1,0,-0.073689,0.370423,-0.106119,-0.007474,...,0,0,0,0,0,0,0,0,0,0
45009,17999,"['harry-potter', 'voldemort']",165000,343000,2,1,-0.148996,0.454998,-0.168628,0.024398,...,0,0,0,0,0,0,0,0,0,0


In [137]:
questions = data_mod[data_mod['num_answer'] == 1]
questions = pd.DataFrame(np.repeat(questions.values, 20, axis=0))
questions.columns = data_mod.columns
questions

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,870,871,872,873,874,875,876,877,878,879
0,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
1,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
2,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
3,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
4,1,"['dc', 'superman', 'character-identification',...",8407,1383,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354895,18000,"['star-trek', 'star-trek-discovery']",141000,598000,8,1,0.00467,0.096009,-0.01663,-0.09926,...,0,0,0,0,0,0,0,0,0,0
354896,18000,"['star-trek', 'star-trek-discovery']",141000,598000,8,1,0.00467,0.096009,-0.01663,-0.09926,...,0,0,0,0,0,0,0,0,0,0
354897,18000,"['star-trek', 'star-trek-discovery']",141000,598000,8,1,0.00467,0.096009,-0.01663,-0.09926,...,0,0,0,0,0,0,0,0,0,0
354898,18000,"['star-trek', 'star-trek-discovery']",141000,598000,8,1,0.00467,0.096009,-0.01663,-0.09926,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#users = pd.read_csv('users.csv', names=['user_num', 'link', 'user_name', 'reputation_sum'], header=None)
#users = users[users['user_num'] <= 20]
#users

In [138]:
users = pd.read_csv('users.csv', names=['user_num', 'link', 'user_name', 'reputation_sum'], header=None)
users = users[users['user_num'] <= 1020]
users = users[users['user_num'] > 1000]
users

,user_num,link,user_name,reputation_sum
1000,1001,https://scifi.stackexchange.com/users/44635/da...,david-says-reinstate-monica,2420
1001,1002,https://scifi.stackexchange.com/users/36849/kaine,kaine,2420
1002,1003,https://scifi.stackexchange.com/users/71305/mo...,motosubatsu,2413
1003,1004,https://scifi.stackexchange.com/users/3007/lus...,luser-droog,2413
1004,1005,https://scifi.stackexchange.com/users/1443/ala...,alan-barber,2411
1005,1006,https://scifi.stackexchange.com/users/751/dkuntz2,dkuntz2,2404
1006,1007,https://scifi.stackexchange.com/users/29081/ja...,jash-jacob,2400
1007,1008,https://scifi.stackexchange.com/users/95390/ju...,just-happen-to-know,2395
1008,1009,https://scifi.stackexchange.com/users/13613/ayrx,ayrx,2394
1009,1010,https://scifi.stackexchange.com/users/89/massimo,massimo,2394


In [139]:
rep = list(users.reputation_sum) * 17745
user_name = list(users.user_name) * 17745
user_num = list(users.user_num) * 17745
print(len(rep), len(user_name))

354900 354900


In [140]:
questions['answer_reputation'] = rep
questions['answer_user_name'] = user_name
questions['user_num'] = user_num
questions

,question_num,tags,question_author_reputation,answer_reputation,answer_likes,num_answer,12,13,14,15,...,872,873,874,875,876,877,878,879,answer_user_name,user_num
0,1,"['dc', 'superman', 'character-identification',...",8407,2420,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,david-says-reinstate-monica,1001
1,1,"['dc', 'superman', 'character-identification',...",8407,2420,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,kaine,1002
2,1,"['dc', 'superman', 'character-identification',...",8407,2413,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,motosubatsu,1003
3,1,"['dc', 'superman', 'character-identification',...",8407,2413,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,luser-droog,1004
4,1,"['dc', 'superman', 'character-identification',...",8407,2411,13,1,-0.215233,0.34735,-0.301892,-0.114613,...,0,0,0,0,0,0,0,0,alan-barber,1005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354895,18000,"['star-trek', 'star-trek-discovery']",141000,2384,8,1,0.00467,0.096009,-0.01663,-0.09926,...,0,0,0,0,0,0,0,0,celestialgranturismo,1016
354896,18000,"['star-trek', 'star-trek-discovery']",141000,2383,8,1,0.00467,0.096009,-0.01663,-0.09926,...,0,0,0,0,0,0,0,0,darth-theory,1017
354897,18000,"['star-trek', 'star-trek-discovery']",141000,2382,8,1,0.00467,0.096009,-0.01663,-0.09926,...,0,0,0,0,0,0,0,0,hendrik-brummermann,1018
354898,18000,"['star-trek', 'star-trek-discovery']",141000,2381,8,1,0.00467,0.096009,-0.01663,-0.09926,...,0,0,0,0,0,0,0,0,kenny,1019


In [141]:
data_train = questions[questions['question_num'] < 1500]
data_test = questions[questions['question_num'] >= 15000]

In [98]:
#data_train = questions[questions['question_num'] < 13000]
#data_test = questions[questions['question_num'] >= 13000]

In [127]:
data_test.answer_user_name

295840               ivo
295841         ummdustry
295842               amy
295843     kevin-workman
295844    pointlessspike
               ...      
354895        anoplexian
354896            darius
354897              user
354898      congusbongus
354899               sjl
Name: answer_user_name, Length: 59060, dtype: object

In [142]:
X_train, X_test = data_train.drop(columns=['tags', 'answer_likes', 'num_answer', 'question_num', 'answer_user_name', 'user_num']), data_test.drop(columns=['tags', 'answer_likes', 'num_answer', 'question_num', 'answer_user_name', 'user_num'])
y_train, y_test = data_train['answer_likes'], data_test['answer_likes']

In [143]:
pred = regressor.predict(X_test)
print(pred)

[0.31640423 0.31640423 0.31640423 ... 0.19405189 0.19405189 0.19405189]


In [144]:
predictions = pd.DataFrame()
predictions['question_num'] = data_test['question_num']
predictions['answer_reputation'] = data_test['answer_reputation']
predictions['user_num'] = data_test['user_num']
predictions['answer_user_name'] = data_test['answer_user_name']
predictions['answer_likes_pred'] = pred
predictions

,question_num,answer_reputation,user_num,answer_user_name,answer_likes_pred
295840,15000,2420,1001,david-says-reinstate-monica,0.316404
295841,15000,2420,1002,kaine,0.316404
295842,15000,2413,1003,motosubatsu,0.316404
295843,15000,2413,1004,luser-droog,0.316404
295844,15000,2411,1005,alan-barber,0.316404
...,...,...,...,...,...
354895,18000,2384,1016,celestialgranturismo,0.194052
354896,18000,2383,1017,darth-theory,0.194052
354897,18000,2382,1018,hendrik-brummermann,0.194052
354898,18000,2381,1019,kenny,0.194052


In [145]:
sorted_pred = predictions.sort_values(by = ['question_num', 'answer_likes_pred'], ascending = [True, False])
sorted_pred

,question_num,answer_reputation,user_num,answer_user_name,answer_likes_pred
295840,15000,2420,1001,david-says-reinstate-monica,0.316404
295841,15000,2420,1002,kaine,0.316404
295842,15000,2413,1003,motosubatsu,0.316404
295843,15000,2413,1004,luser-droog,0.316404
295844,15000,2411,1005,alan-barber,0.316404
...,...,...,...,...,...
354895,18000,2384,1016,celestialgranturismo,0.194052
354896,18000,2383,1017,darth-theory,0.194052
354897,18000,2382,1018,hendrik-brummermann,0.194052
354898,18000,2381,1019,kenny,0.194052


In [146]:
sorted_pred['num_answer_pred'] = 0
sorted_pred.reset_index(drop=True, inplace=True)
sorted_pred

,question_num,answer_reputation,user_num,answer_user_name,answer_likes_pred,num_answer_pred
0,15000,2420,1001,david-says-reinstate-monica,0.316404,0
1,15000,2420,1002,kaine,0.316404,0
2,15000,2413,1003,motosubatsu,0.316404,0
3,15000,2413,1004,luser-droog,0.316404,0
4,15000,2411,1005,alan-barber,0.316404,0
...,...,...,...,...,...,...
59055,18000,2384,1016,celestialgranturismo,0.194052,0
59056,18000,2383,1017,darth-theory,0.194052,0
59057,18000,2382,1018,hendrik-brummermann,0.194052,0
59058,18000,2381,1019,kenny,0.194052,0


In [147]:
q_num = 0
for i in range (59060):
    if sorted_pred['question_num'][i] != q_num:
        q_num = sorted_pred['question_num'][i]
        sorted_pred['num_answer_pred'][i] = 1

Выходные данные были обрезаны до нескольких последних строк (5000).
<ipython-input-147-c61cb7f4ef27>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_pred['num_answer_pred'][i] = 1
<ipython-input-147-c61cb7f4ef27>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_pred['num_answer_pred'][i] = 1
<ipython-input-147-c61cb7f4ef27>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sorted_pred['num_answer_pred'][i] = 1
<ipython-in

In [134]:
a = sorted_pred[sorted_pred['num_answer_pred'] == 1]
a

,question_num,answer_reputation,user_num,answer_user_name,answer_likes_pred,num_answer_pred
0,15000,5271,502,ummdustry,0.330978,1
20,15001,5271,502,ummdustry,0.183892,1
40,15002,5284,501,ivo,0.458313,1
60,15003,5284,501,ivo,0.356887,1
80,15004,5284,501,ivo,0.281462,1
...,...,...,...,...,...,...
58960,17996,5271,502,ummdustry,0.258800,1
58980,17997,5271,502,ummdustry,0.233989,1
59000,17998,5271,502,ummdustry,0.172929,1
59020,17999,5271,502,ummdustry,0.228666,1


In [135]:
a['user_num'].value_counts()

user_num
502    1953
501    1000
Name: count, dtype: int64

In [136]:
a['answer_user_name'].value_counts()

answer_user_name
ummdustry    1953
ivo          1000
Name: count, dtype: int64

In [83]:
a['answer_reputation'].value_counts()

answer_reputation
102634    1790
103728     569
116695     516
119235      37
131516      11
111159       8
97671        8
69933        7
82018        5
107461       1
105914       1
Name: count, dtype: int64

In [109]:
ans = pd.DataFrame(a['answer_reputation'].value_counts())
ans_u = pd.DataFrame(a['answer_user_name'].value_counts())
ans_n = pd.DataFrame(a['user_num'].value_counts())

In [113]:
ans_n

,count
user_num,
6,2186
1,729
13,14
4,9
5,9
2,2
17,2
19,2


In [115]:
result = pd.DataFrame()
result['num'] = ans_n.index
result['name'] = ans_u.index
result['reputation'] = ans.index
result['count'] = 0
for i in range (8):
    result['count'][i] = ans['count'][i]
result

<ipython-input-115-1e61e3b630d3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['count'][i] = ans['count'][i]
<ipython-input-115-1e61e3b630d3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['count'][i] = ans['count'][i]
<ipython-input-115-1e61e3b630d3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['count'][i] = ans['count'][i]
<ipython-input-115-1e61e3b630d3>:7: SettingWithCopyWarning: 
A value is trying to 

,num,name,reputation,count
0,6,user14111,164192,2186
1,1,valorum,695009,729
2,13,adamant,116695,14
3,4,thaddeus-howze,212888,9
4,5,slytherincess,164934,9
5,2,dvk-on-ahch-to,342931,2
6,17,john-rennie,107461,2
7,19,jeff,105914,2


In [116]:
result

,num,name,reputation,count
0,6,user14111,164192,2186
1,1,valorum,695009,729
2,13,adamant,116695,14
3,4,thaddeus-howze,212888,9
4,5,slytherincess,164934,9
5,2,dvk-on-ahch-to,342931,2
6,17,john-rennie,107461,2
7,19,jeff,105914,2


In [91]:
result['count'][10] = 1
result

<ipython-input-91-efe5df6d1fcb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['count'][10] = 1


,num,name,reputation,count
0,22,alexwlchan,102634,1790
1,21,kutulumike,103728,569
2,13,adamant,116695,516
3,12,logicdictates,119235,37
4,11,davidw,131516,11
5,16,praxis,111159,8
6,23,buzz,97671,8
7,28,nel,69933,7
8,26,ancientswordrage,82018,5
9,17,john-rennie,107461,1


In [55]:
result

,name,reputation,count
0,alexwlchan,102634,893
1,kutulumike,103728,300
2,adamant,116695,252
3,logicdictates,119235,17
4,praxis,111159,4
5,davidw,13151,3
6,buzz,97671,3
7,ancientswordrage,82018,2


In [136]:
a['answer_reputation'].value_counts()

answer_reputation
103728    2587
107461     281
111159      33
116695      28
114920      15
119235       5
105914       4
Name: count, dtype: int64

In [ ]:
a['answer_reputation'].value_counts()

answer_reputation
695009    125
107461      2
164192      2
Name: count, dtype: int64

In [ ]:
a['answer_reputation'].value_counts()

answer_reputation
131516    152
107461     61
105914     35
119235      6
114920      2
111159      1
116695      1
Name: count, dtype: int64

# Нейронки

In [51]:
from keras import models
from keras import layers

In [52]:
model = models.Sequential()
model.add(layers.Dense(870, activation='relu'))
model.add(layers.Dense(435, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [53]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=42,
                    validation_data=(X_test, y_test))

Epoch 1/20
893/893 [==============================] - 27s 24ms/step - loss: 1.9445 - accuracy: 0.6051 - val_loss: 0.6700 - val_accuracy: 0.6080
Epoch 2/20
893/893 [==============================] - 20s 22ms/step - loss: 0.6709 - accuracy: 0.6053 - val_loss: 0.6696 - val_accuracy: 0.6080
Epoch 3/20
893/893 [==============================] - 21s 24ms/step - loss: 0.6708 - accuracy: 0.6053 - val_loss: 0.6696 - val_accuracy: 0.6080
Epoch 4/20
893/893 [==============================] - 22s 25ms/step - loss: 0.6708 - accuracy: 0.6053 - val_loss: 0.6696 - val_accuracy: 0.6080
Epoch 5/20
893/893 [==============================] - 22s 24ms/step - loss: 0.6708 - accuracy: 0.6053 - val_loss: 0.6696 - val_accuracy: 0.6080
Epoch 6/20
893/893 [==============================] - 22s 24ms/step - loss: 0.6708 - accuracy: 0.6053 - val_loss: 0.6697 - val_accuracy: 0.6080
Epoch 7/20
893/893 [==============================] - 21s 24ms/step - loss: 0.6708 - accuracy: 0.6053 - val_loss: 0.6696 - val_accuracy:

In [33]:
import keras
from keras import models
from keras import layers

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

In [36]:
import tensorflow as tf

In [39]:
X_train

,question_author_reputation,answer_reputation,12,13,14,15,16,17,18,19,...,870,871,872,873,874,875,876,877,878,879
0,8407,1383,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,...,0,0,0,0,0,0,0,0,0,0
1,8407,633000,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,...,0,0,0,0,0,0,0,0,0,0
2,8407,1,-0.215233,0.347350,-0.301892,-0.114613,0.193532,-0.134682,0.383736,0.085273,...,0,0,0,0,0,0,0,0,0,0
3,108000,695000,0.003884,0.167487,-0.091520,0.130714,-0.096572,-0.141100,0.386765,-0.116209,...,0,0,0,0,0,0,0,0,0,0
4,108000,31,0.003884,0.167487,-0.091520,0.130714,-0.096572,-0.141100,0.386765,-0.116209,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37472,331000,104000,0.038582,0.309145,-0.267616,0.263565,-0.045215,-0.003271,0.224649,0.020339,...,0,0,0,0,0,0,0,0,0,0
37473,331000,605000,0.038582,0.309145,-0.267616,0.263565,-0.045215,-0.003271,0.224649,0.020339,...,0,0,0,0,0,0,0,0,0,0
37474,1614,165000,-0.121199,0.311127,-0.234067,0.069905,0.054047,-0.190292,0.421074,0.094105,...,0,0,0,0,0,0,0,0,0,0
37475,1614,41,-0.121199,0.311127,-0.234067,0.069905,0.054047,-0.190292,0.421074,0.094105,...,0,0,0,0,0,0,0,0,0,0


In [40]:
model = keras.Sequential([
    layers.Dense(870, activation='relu'),
    layers.Dense(1200, activation='relu'),
    layers.Dense(400, activation='relu'),
    layers.Dropout(0.1),
    layers.Dense(400, activation='relu'),
    layers.Dense(200, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(12, activation='relu'),
    layers.Dense(2)])

optimizer = tf.optimizers.Adam()
model.compile(loss=tf.keras.losses.MSE, optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=20)

Epoch 1/20
1172/1172 [==============================] - 51s 41ms/step - loss: 119549.3594 - accuracy: 0.5048
Epoch 2/20
1172/1172 [==============================] - 48s 41ms/step - loss: 1.7600 - accuracy: 0.4483
Epoch 3/20
1172/1172 [==============================] - 48s 41ms/step - loss: 0.2596 - accuracy: 0.4252
Epoch 4/20
1172/1172 [==============================] - 47s 40ms/step - loss: 0.3046 - accuracy: 0.4200
Epoch 5/20
1172/1172 [==============================] - 47s 40ms/step - loss: 0.2889 - accuracy: 0.4512
Epoch 6/20
1172/1172 [==============================] - 47s 40ms/step - loss: 0.2467 - accuracy: 0.5027
Epoch 7/20
1172/1172 [==============================] - 46s 39ms/step - loss: 0.2455 - accuracy: 0.5120
Epoch 8/20
1172/1172 [==============================] - 48s 41ms/step - loss: 0.2286 - accuracy: 0.4953
Epoch 9/20
1172/1172 [==============================] - 48s 41ms/step - loss: 0.3123 - accuracy: 0.4999
Epoch 10/20
1172/1172 [==============================] - 48